# download MERGE_IR half-hourly images, a bigger equatorial box, before Irma 
## And then make movie frames of it

Restart stalled kernel and it will take up where it left off. Delete outputs to start over. 

In [1]:
import os 
import numpy as np
import xarray as xr
from datetime import date, timedelta, datetime
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from matplotlib.image import imread
import geocat.viz.util as gvutil
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from collections import deque
import matplotlib.patches as patches
from pathlib import Path
import json

In [2]:
class EyedropperAnnotator:
    def __init__(self):
        self.brown_events = deque()
        
    def add_events(self, centroids, sizes, current_time):
        """Add new events with sizes at current time"""
        for (lat, lon), size in zip(centroids, sizes):
            self.brown_events.append((lon, lat, current_time, size))
    
    def save_state(self, filepath):
        """Overwrite current state (for restart)"""
        def convert_event(event):
            return tuple(float(x) if isinstance(x, (np.floating, np.integer)) else x for x in event)
        
        state = {'brown_events': [convert_event(e) for e in self.brown_events]}
        with open(filepath, 'w') as f:
            json.dump(state, f)

    def append_history(self, filepath):
        """Append all events to growing file"""
        def convert_event(event):
            return tuple(float(x) if isinstance(x, (np.floating, np.integer)) else x for x in event)
        
        snapshot = {'brown_events': [convert_event(e) for e in self.brown_events]}
        with open(filepath, 'a') as f:
            f.write(json.dumps(snapshot) + '\n')

    @staticmethod
    def load_state(filepath):
        """Load state for restart, or create new if file doesn't exist"""
        if not Path(filepath).exists():
            return EyedropperAnnotator()
        with open(filepath, 'r') as f:
            state = json.load(f)
        annotator = EyedropperAnnotator()
        annotator.brown_events = deque(state['brown_events'])
        return annotator


    def plot_one_expanding_ring(self, ax, current_time, facecolor='orange'):
        """Plot expanding annuli with size-dependent opacity"""
        import matplotlib.patches as patches
        import cartopy.crs as ccrs
        
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age * 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2)
            alpha = min(0.8, size / max(area, 1))
            
            if alpha > 0.01:
                if outer_r > 0:
                    annulus = patches.Annulus((lon, lat), outer_r, width=outer_r/2, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]

    def plot_expanding_ring_coriolis_pinwheel(self, ax, current_time, facecolor='orange', 
                                     spoke_color='white', n_spokes=12, mute=1):
        """
        Plot expanding annuli with Coriolis-curved spokes.
        
        The spokes show rotation proportional to Coriolis force.        
        Args:
            ax: matplotlib axis
            current_time: current frame time
            facecolor: color of annulus
            spoke_color: color of Coriolis spokes
            n_spokes: number of spokes (default 12)
            mute: a factor to multiply all alphas by 
        """
        
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time  # seconds 
            if age < 0:
                continue
                
            outer_r = age    * 52 * 60 * 30 / 111111.1  # degrees
            inner_r = (age-1)* 52 * 60 * 30 / 111111.1
            width =         1* 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2 - inner_r**2)
            alpha = min(0.8, size / max(area, 1))*mute  # min and max keep it in [0,0.8] 
            
            if alpha > 0.01:
                if outer_r > 0:
                    # Draw base annulus
                    annulus = patches.Annulus((lon, lat), outer_r, width=width, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)


                    # Draw Coriolis spokes for this age 
                    lat_rad = np.radians(lat)
                    f = 2*2*np.pi/86400. *np.sin(lat_rad)  # Coriolis , f(in /s, lat)
                    
                    # Mean counterclockwise rotation increases with age for f>0
                    # If a ring of parcels contracts by a radial distance dr, 
                    # the area of the contraction annulus is set equal to size. 
                    # then vtan = f*dr, distance traversed is dxtan = f*dr*age, 
                    # dtheta = dxtan / (2*pi*r)
 
                    
                    for spoke_idx in range(n_spokes):
                        # Base azimuth for this spoke at outer_r 
                        base_azimuth = np.radians(spoke_idx * 360 / n_spokes)
                        
                        # Create curved spoke from outer_r inward to radius of mass sink (size)
                        n_points = 50
                        radii = np.linspace(outer_r, np.sqrt(size/np.pi), n_points) 

                        dr = size /2/np.pi/radii *111111.   # meters 
                        vtan = f*dr      # m/s, function of latitude
                        dxtan= vtan*age                     # meters 
                        dtheta = dxtan /2/np.pi/radii       # radians
                        
                        # Azimuth(radii), varies along spoke due to Coriolis
                        azimuths = base_azimuth + dtheta 
                        
                        # Convert to lon/lat offsets
                        x_offsets = radii * np.cos(azimuths)
                        y_offsets = radii * np.sin(azimuths)
                        
                        # Create curved path
                        spoke_lons = lon + x_offsets
                        spoke_lats = lat + y_offsets
                        
                        # Plot as curved line
                        ax.plot(spoke_lons, spoke_lats, color=spoke_color, \
                                transform=ccrs.PlateCarree(), \
                                alpha = alpha)   # min(0.9, 1 / max(area, 1)))   
                        
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]    

In [3]:
# WATERSHED get_feature_centroids_and_sizes, sizes in pixels (data_array, sigma=5, threshold=230, min_size=0)

from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt
from scipy.ndimage import label, gaussian_filter

def get_feature_centroids_and_sizes(data_array, sigma=5, threshold=230, min_size=0):
    smimage = gaussian_filter(data_array.values, sigma=sigma)
    binary = smimage < threshold
    
    dist = distance_transform_edt(binary)
    labeled_array = watershed(-dist, mask=binary)
    
    lats = data_array.lat.values
    lons = data_array.lon.values
    
    centroids, sizes = [], []
    for fid in range(1, labeled_array.max() + 1):
        rows, cols = np.where(labeled_array == fid)
        size = len(rows)
        if size >= min_size:
            sizes.append(size)
            centroids.append((lats[int(rows.mean())], lons[int(cols.mean())]))
    
    return centroids, sizes  

#from matplotlib.patches import Circle  # for a graphical test 
#centroids,sizes = get_feature_centroids_and_sizes(data_array, sigma=5, threshold=230)
#data_array.plot()
#for (lat, lon), size in zip(centroids, sizes):
#    plt.gca().add_patch(Circle((lon, lat), radius=np.sqrt(size/200/np.pi), fill=False, edgecolor='red'))

# okay let's make the frames of animation 

In [4]:
# Open data files 

from pathlib import Path
datafiles = '/Volumes/Samsung USB/PrePreIrma_IRfiles/'+ \
            'merg_2017*_4km-pixel.nc4'

OUTPUT = Path('/Volumes/Samsung USB/PrePreIrma_annot_spokes/')
OUTPUT.mkdir(exist_ok=True, parents=True) 
OUTDIR = '/Volumes/Samsung USB/PrePreIrma_annot_spokes/'

# OUTPUT = Path('~/Downloads/PrePreIrma_annot_spokes/')
# OUTPUT.mkdir(exist_ok=True, parents=True) 
# OUTDIR = '~/Downloads/PrePreIrma_annot_spokes/'

ds = xr.open_mfdataset(datafiles)
Tb = ds.Tb
Tb

<xarray.DataArray 'Tb' (time: 314, lat: 1100, lon: 1374)> Size: 2GB
dask.array<concatenate, shape=(314, 1100, 1374), dtype=float32, chunksize=(2, 1100, 1374), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 5kB 0.01819 0.05457 0.09094 ... 49.89 49.93 49.97
  * lat      (lat) float32 4kB -19.99 -19.96 -19.92 -19.88 ... 19.92 19.96 19.99
  * time     (time) datetime64[ns] 3kB 2017-08-13T05:00:00.000013312 ... 2017...
Attributes:
    units:          K
    standard_name:  brightness_temperature

In [5]:
# panzoom function Interpolates "extent" as a function of frame_count)

# Define interpolation function with optional time warping
def panzoom(frame_count, frame_start, frames_total, easing=None):
    if frame_count <= frame_start:
        return extent
    if frame_count >= frame_end:
        return extent2
    # Normalize t to [0, 1]
    t = (frame_count - frame_start) / frames_total
    # Optionally apply non-linear time mapping
    if easing:
        t = easing(t)
    return [np.interp(t, [0, 1], [e1, e2]) for e1, e2 in zip(extent, extent2)]

In [6]:
# UPDATED create_masked_cmap function: a time dependent colormap with black masked range of Tb 
# time it is its parameter, retreating from the cold end except the orange is there too 

from matplotlib.colors import LinearSegmentedColormap, to_rgba
import numpy as np

def create_masked_cmap(it, vmin=190, vmax=340, verbose=False, 
                       mask_bot=220, mask_top=305, blackrate=1.0):
    """
    Create a colormap with a time-dependent retreating black mask.
    
    Args:
        it: time parameter (frame number or time index)
        vmin, vmax: data value range
        verbose: print mask range info
        mask_bot: bottom of mask band (retreats with time)
        mask_top: top of mask band (fixed)
        blackrate: how fast the mask retreats (Tb units per time step)
    
    Returns:
        cmap: LinearSegmentedColormap with masked band
    """
    
# Brigher colors from BlueMarble (less white haze at high T), less garish hot orange
    color_points = [
        (190, "#dc05ef", 1.0),  # magenta 
        (222, "#0589ef", 1.0),  # blue (color enhancement ends)
        (240, "#00ffff", 1.0),  # cyan
        (250, "#716f6f", 1.0),  # darker gray 
        (270, "#c5c6c6", 1.0),  # light-mid gray
        (280, "#ffffff", 0.7),  # white semitrans light gray
        (290, "#ffffff", 0.3),  # white point surface
        (300, "#ffffff", 0.1),  # SST point water
        (305, "#ffffff", 0.3),  # --> WHITE 
        (310, "#ff8000", 0.6),  # HOT surface orange, see https://html-color.codes/
        (340, "#000000", 0.8)  # HOTHOT surface black
    ]
        
    # Create base colormap first
    cmap_colors = [((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points]
    base_cmap = LinearSegmentedColormap.from_list("base_custom", cmap_colors)
    
    # Calculate mask band: [mask_bot + it*blackrate, mask_top]
    mask_lower = mask_bot + it * blackrate
    mask_upper = mask_top
    mask_lower_norm = (mask_lower - vmin) / (vmax - vmin)
    mask_upper_norm = (mask_upper - vmin) / (vmax - vmin)
    
    if verbose:
        print('mask range: ', mask_lower, mask_upper)
    
    # Sample the interpolated colormap densely, then mask it smoothly
    n_samples = 256
    pos_array = np.linspace(0, 1, n_samples)
    rgba_array = base_cmap(pos_array)
    
    # Set masked band to black with full opacity
    mask_idx = (pos_array >= mask_lower_norm) & (pos_array <= mask_upper_norm)
    rgba_array[mask_idx] = [0, 0, 0, 1.0]
    
    # Create new colormap from the masked array
    return LinearSegmentedColormap.from_list("masked_custom", 
                                             list(zip(pos_array, rgba_array)))

# Usage:
# cmap = create_masked_cmap(it=10, blackrate=1.0, verbose=True)
# colored = cmap(normalized_data)

In [7]:
# Read in big Blue Marble image 
# --- Place this code BEFORE your 1000-frame loop (Load ONCE) ---

# Disable the decompression bomb check
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

# --- Configuration (MUST match your image and desired georeference) ---
STATIC_MAP_FILE = '/Users/bmapes/Box/Sky_Symphony_Box/BlueMarble.200408.3x21600x10800.jpg' 
FULL_WORLD_EXTENT = [-180, 180, -90, 90] 
try:
    # Load the single satellite image file ONLY ONCE
    BASE_IMAGE_DATA = imread(STATIC_MAP_FILE)
except FileNotFoundError:
    raise FileNotFoundError(f"Error: Static map file '{STATIC_MAP_FILE}' not found. '+\
    'Please download a high-res Blue Marble image and save it to this name.")

In [8]:
# Loop over times, making frames in the named folder frame_folder 

vmin = 190; vmax=340 # as in Irma movie frames
dpi = 150

mask_bottom = 220  # 50th frame is wonderful at 250 
blackrate = 0.4 # mask reveal rate, K per frame. 1 starts grays at frame 50/313, maybe 0.3 so ~half is abstract blue tops 

frame_start = 250 # Start pan-zooming at frame 250
frame_end   = 350 # Done pan-zooming at frame 250

# Initialize annotator, loading past state if this is a restart
# annotator = EyedropperAnnotator.load_state('annotator_state.pkl')
# annotator = EyedropperAnnotator( )  

# Define the end extent of this frame sequence, for panzoom function 
# That is, the first extent of the regular Irma movie 
ds2 = xr.open_dataset("/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/ncfiles/time000.nc")
extent2 = [ min(ds2.lon), max(ds2.lon),
            min(ds2.lat), max(ds2.lat) ]

# Loop over the times, making the frames
for frame_count in range(len(ds.time)): 
    
    ######## Plotting code if file does not exist (for restart after kernel crashes) 
    out_file = OUTDIR + f"{frame_count:08d}.png"
    if not os.path.exists(out_file):

        # On restart:
        annotator = EyedropperAnnotator().load_state('annotator_state.pkl')

        Tb = ds.Tb.isel(time = frame_count).interpolate_na(dim="lon", method="linear", max_gap=2)

# Convective Events 
        centroids,sizes = get_feature_centroids_and_sizes(Tb, sigma=7, threshold=230, 
                                                      min_size=200)  # pixels 
        sizes = np.array(sizes) * (4/111.)**2 # square degrees from 4km pixels
        # print('sizes ', sizes)
    # Define extent from the data, and extent_now as a function of frame_count
    # interpolating toward the first frame of Irma... 
        extent = [ min(Tb.lon), max(Tb.lon), -16, 16 ]
                   # min(Tb.lat), max(Tb.lat) ] # aspect ratio was wrong in data, gotta match movie_IRMA

    #### Time dependent effects
    # Start zooming at frame 250
        # frame_start = 250    # set above 
        frame_end = len(ds.time) - 1  # assuming 0-based indexing
        frames_total = frame_end - frame_start
        extent_now = panzoom(frame_count, frame_start, frames_total, easing=False)
    
    # Fade from black effect: unveiling from coldest to warmest, but all with hot orange daytime showing
        cmap = create_masked_cmap(it=frame_count, vmin=vmin, vmax=vmax, verbose=False, 
                                  mask_bot = mask_bottom, mask_top = 310, blackrate=blackrate) 
                                                  # blackrate in K per timestep (1/2 hour)
        
            
    # Plot data on top of blue marble image, there is some transparency 
        fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Extent sets the area covered.
    # pan+zoom effect has extent transition from full to Irma-begin coordinates. 
        ax.set_extent(extent_now, crs=ccrs.PlateCarree())
        
    # Background image: cheap stock image? Naw, blue marble for August
        #ax.stock_img()  # low-res natural Earth background
        ax.imshow(BASE_IMAGE_DATA,      
          origin='upper', transform=ccrs.PlateCarree(), extent=FULL_WORLD_EXTENT)
    
        ax.coastlines()
            #ax.coastlines(resolution='50m', color='white', linewidth=0.5, zorder=3)
            #ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.set_title(f"Tb at {Tb.time.values}", fontsize=12)
    
    # image of the data   
        Tb.plot.imshow(
            ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
            add_colorbar=False, #cbar_kwargs={'label': 'Brightness Temperature [K]'}
        )
        plt.tight_layout()
        
# Add eyedropper annotations    
        # In your loop:
        annotator.add_events(centroids, (np.array(sizes)).tolist(), frame_count)
        
        mute = max(0, 1 - (frame_count - frame_start) / (frame_end - frame_start))
        annotator.plot_expanding_ring_coriolis_pinwheel(ax, frame_count, 
                                       facecolor='orange', 
                                       spoke_color='white',
                                       n_spokes=8, mute=mute)

        annotator.save_state('annotator_state.pkl')
        annotator.append_history('annotator_history.jsonl')
        # Might want to start fading annotations during panzoom 
        if(frame_count == frame_start):
            annotator.save_state('annotator_state.PANZOOMSTART.pkl')
        if(frame_count == 200):
            annotator.save_state('annotator_state.200.pkl')
                
# Cover annotations with zorder=10, near the souce where they are too hokey 
        Tb.where(Tb < 230).plot.imshow(ax=ax, add_colorbar=False, cmap=cmap, vmin=vmin,vmax=vmax, zorder=10)
        
        plt.savefig(out_file, dpi=dpi)
        plt.close()
        print(f"Saved {out_file}")
        print(f"Tb at {Tb.time.values}")

    

Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000251.png
Tb at 2017-08-18T10:30:00.000000000
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000252.png
Tb at 2017-08-18T11:00:00.000013312
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000253.png
Tb at 2017-08-18T11:30:00.000026880
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000254.png
Tb at 2017-08-18T12:00:00.000000000
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000255.png
Tb at 2017-08-18T12:30:00.000013312
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000256.png
Tb at 2017-08-18T13:00:00.000026880
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000257.png
Tb at 2017-08-18T13:30:00.000000000
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000258.png
Tb at 2017-08-18T14:00:00.000013312
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000259.png
Tb at 2017-08-18T14:30:00.000026880
Saved /Volumes/Samsung USB/PrePreIrma_annot_spokes/00000260.png
Tb at 2017-08-18T15:00:00.000000000


In [9]:
mute

0

In [10]:
# Crop them all 

from pathlib import Path; from PIL import Image
SOURCE = Path('/Volumes/Samsung USB/PrePreIrma_annot_spokes/')
OUTPUT = Path('/Volumes/Samsung USB/PrePreIrma_pinwheels_crop/')
OUTPUT.mkdir(exist_ok=True) 

CROP = (210, 70, 1290, 850)
for f in SOURCE.glob('*.png'): Image.open(f).crop(CROP).save(OUTPUT / f.name)

In [11]:
# 4. Create simple video using ffmpeg
from subprocess import run

ffmpeg_cmd = [
    "ffmpeg", "-y", "-framerate", "3",
    "-i", str(OUTPUT / "%08d.png"),
    "-c:v", "libx264", "-pix_fmt", "yuv420p",
    "/Volumes/Samsung USB/PrePreIrma_pinwheels.mp4"
]

print("Creating video...")
run(ffmpeg_cmd)
print(f"✅ Video saved")

Creating video...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/bmapes/.local/share/mamba/envs/hk25 --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1746479731466/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libmp3lame --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zli

✅ Video saved


[out#0/mp4 @ 0x600002944000] video:50952KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.008142%
frame=  314 fps= 45 q=-1.0 Lsize=   50956KiB time=00:01:44.00 bitrate=4013.8kbits/s speed=  15x    
[libx264 @ 0x143705aa0] frame I:12    Avg QP:15.25  size:199214
[libx264 @ 0x143705aa0] frame P:159   Avg QP:20.60  size:166734
[libx264 @ 0x143705aa0] frame B:143   Avg QP:21.16  size:162746
[libx264 @ 0x143705aa0] consecutive B-frames: 37.9%  2.5%  4.8% 54.8%
[libx264 @ 0x143705aa0] mb I  I16..4: 13.7% 46.5% 39.8%
[libx264 @ 0x143705aa0] mb P  I16..4:  0.5% 28.3% 30.0%  P16..4: 11.7% 13.8% 10.3%  0.0%  0.0%    skip: 5.3%
[libx264 @ 0x143705aa0] mb B  I16..4:  0.2% 15.4% 31.1%  B16..8: 15.4% 14.7%  9.0%  direct:11.6%  skip: 2.6%  L0:31.7% L1:26.7% BI:41.6%
[libx264 @ 0x143705aa0] 8x8 transform intra:42.1% inter:60.2%
[libx264 @ 0x143705aa0] coded y,uvDC,uvAC intra: 96.6% 96.6% 89.1% inter: 81.4% 82.1% 44.5%
[libx264 @ 0x143705aa0] i16 v,h,dc,p: 63% 10% 1

In [15]:
# ffmpeg -f concat -safe 0 -i <(echo -e "file 'input1.mp4'\nfile 'input2.mp4'") -c copy output.mp4

!ffmpeg -f concat -safe 0 -i <(echo -e "file \
'/Users/bmapes/Box/Sky_Symphony_Box/PrePreIrma_pinwheels.mp4'\n file \
'/Users/bmapes/Box/Sky_Symphony_Box/Irma_bmc_8xframes_framerate24.mp4'") -c copy \
/Users/bmapes/Box/Sky_Symphony_Box/Irma_Wholeshow.mp4



ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/bmapes/.local/share/mamba/envs/hk25 --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1746479731466/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libmp3lame --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zli

In [ ]:
# video: https://miami.box.com/s/xmjenndyx5ktzzdcooj0a662lmrdxk0h